In [1]:
import torch
import pickle, gzip
import sys
import numpy as np
import pandas as pd
import os
from voc import read_object_labels_csv

/home/seongha/anaconda3/envs/MGSSL/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
coco_lt_path = "data/coco"
voc_lt_path = "data/voc/"

# VOC2007

In [4]:
path_csv = './data/voc/files/VOC2007'
trainval_csv = os.path.join(path_csv, 'classification_trainval.csv')
test_csv = os.path.join(path_csv, 'classification_test.csv')

In [5]:
import pandas as pd

In [6]:
hi = pd.read_csv(trainval_csv)
print(hi.columns)
print(hi.shape)



Index(['name', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car',
       'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
       'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'],
      dtype='object')
(5011, 21)


In [6]:
hi = pd.read_csv(trainval_csv)
print(hi.columns)
print(hi.shape)
## generate gt pkl file
hi.replace(-1,0)
gt_labels = hi.iloc[:,1:].to_numpy()
print(gt_labels.shape)
img_id2idx = dict()
idx2img_id = dict()

for i, row in hi.iterrows():
  img_id2idx[row['name']] = i
  idx2img_id[i] = row['name']

print(len(img_id2idx), len(idx2img_id))

Index(['name', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car',
       'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
       'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'],
      dtype='object')
(1058, 21)
(1058, 20)
1058 1058


In [10]:
import pickle

In [32]:
res = {
  "gt_labels": gt_labels,
  "img_id2idx": img_id2idx,
  "idx2img_id": idx2img_id
}
with open('terse_gt_2007.pkl', 'wb') as f:
  pickle.dump(res, f)

In [10]:
di = dict()
for col_name, val in hi.iteritems():
  print(col_name)
  if col_name =='name':
    continue
  di[col_name] = val.value_counts().to_dict()

import json
with open("voc_class_num.json", 'w') as f:
  json.dump(di, f)


name
aeroplane
bicycle
bird
boat
bottle
bus
car
cat
chair
cow
diningtable
dog
horse
motorbike
person
pottedplant
sheep
sofa
train
tvmonitor


In [33]:
with open(voc_lt_path + "img_id.txt", "r") as f:
  # data= pickle.load(f)
  data = f.readlines()
  # print(data)
  data = [int(each) for each in data]
  # data = list(map(str,data))
  print(len(data))
print(max(data))
# print(hi['name'].isin({'name': data}))
# print(hi['name'].dtype)

LT = hi.loc[hi['name'].isin(data)]

LT['name'] = LT['name'].apply(lambda x:format(x, '06d'))
print(LT['name'])
import os
LT_path = os.path.join(path_csv, 'classification_trainval_' + "LT.csv")
LT.to_csv(LT_path, index=False) 

1123
9962
Series([], Name: name, dtype: int64)


# COCO2014

In [13]:
path_csv = './data/coco/'
import json
with open('data/coco/data/category.json') as f:

  category = json.load(f)
  print(category.keys())
  print(len(category.items()))
  print(category.values())

with open('data/coco/data/train_anno.json') as f:

  train = json.load(f)
  print(len(train), train[0])

  li = []
  gt_labels = np.zeros((len(train),80))
  img_id2idx = dict()
  idx2img_id = []
  for i,each in enumerate(train):
    li += each['labels']
    gt_labels[i, each['labels']] = 1
    img_id = int(each['file_name'].split('.')[0].split('_')[-1])
    idx2img_id.append(img_id)
    img_id2idx[img_id] = i
  print(max(li))

  df = pd.DataFrame(gt_labels)
  print(df.columns)
  di = dict()
  for col_name, val in df.iteritems():
    if col_name == 'name': continue
    print(val)
    df[col_name] = val.value_counts().to_dict()
  
  with open('coco_class_num', 'w') as f:
    json.dump(di, f)

with open('data/coco/data/val_anno.json') as f:

  val = json.load(f)
  print(len(val), val[0])

  test_gt_labels = np.zeros((len(val),80))
  
  for i,each in enumerate(val):
    test_gt_labels[i, each['labels']] = 1
  print(test_gt_labels)

dict_keys(['airplane', 'apple', 'backpack', 'banana', 'baseball bat', 'baseball glove', 'bear', 'bed', 'bench', 'bicycle', 'bird', 'boat', 'book', 'bottle', 'bowl', 'broccoli', 'bus', 'cake', 'car', 'carrot', 'cat', 'cell phone', 'chair', 'clock', 'couch', 'cow', 'cup', 'dining table', 'dog', 'donut', 'elephant', 'fire hydrant', 'fork', 'frisbee', 'giraffe', 'hair drier', 'handbag', 'horse', 'hot dog', 'keyboard', 'kite', 'knife', 'laptop', 'microwave', 'motorcycle', 'mouse', 'orange', 'oven', 'parking meter', 'person', 'pizza', 'potted plant', 'refrigerator', 'remote', 'sandwich', 'scissors', 'sheep', 'sink', 'skateboard', 'skis', 'snowboard', 'spoon', 'sports ball', 'stop sign', 'suitcase', 'surfboard', 'teddy bear', 'tennis racket', 'tie', 'toaster', 'toilet', 'toothbrush', 'traffic light', 'train', 'truck', 'tv', 'umbrella', 'vase', 'wine glass', 'zebra'])
80
dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3

ValueError: Length of values (2) does not match length of index (82081)

In [17]:
with open('data/coco/data/train_anno_LT.json') as f:

  train = json.load(f)
  print(len(train), train[0])

  li = []
  gt_labels = np.zeros((len(train),80))
  img_id2idx = dict()
  idx2img_id = []
  for i,each in enumerate(train):
    li += each['labels']
    gt_labels[i, each['labels']] = 1
    img_id = int(each['file_name'].split('.')[0].split('_')[-1])
    idx2img_id.append(img_id)
    img_id2idx[img_id] = i

  print(gt_labels.sum(axis=0))
  np.save('coco_class_num.npy', gt_labels.sum(axis=0))
  
  

  
  with open('coco_class_num', 'w') as f:
    json.dump(di, f)

1861 {'file_name': 'COCO_train2014_000000457732.jpg', 'labels': [13, 14, 22, 57, 26, 27]}
[  10.   34.   85.   39.   24.   18.    6.   49.   68.   44.   21.   17.
  297.  442.  364.   32.   32.   75.  251.   25.   55.  113.  578.   92.
  177.   11.  445.  646.   71.   15.    8.   14.  102.   11.    7.    6.
  175.   17.   13.   75.   10.  183.  131.   92.   29.   71.   33.  168.
    8. 1098.   41.  152.  131.   89.   44.   15.    7.  258.   18.   12.
    8.  162.   34.   14.   20.    9.   20.   20.   35.    9.   24.   12.
   47.   18.   90.  207.   50.  118.   97.    7.]


In [50]:
res = {
  "gt_labels": gt_labels,
  "img_id2idx": img_id2idx,
  "idx2img_id": idx2img_id,
  "test_gt_labels": test_gt_labels
}
with open('terse_gt_2014.pkl', 'wb') as f:
  pickle.dump(res, f)

In [64]:
with open("data/coco/img_id.pkl", "rb") as f:
  # data= pickle.load(f)
  data = f.readlines()
  data = [int(each) for each in data]
  # print(data)
  train_anno = []
  with open('data/coco/data/train_anno.json') as f:

    train = json.load(f)
    print(len(train), train[0])
    for each in train:
      img_id = int(each['file_name'].split('.')[0].split('_')[-1])
      if img_id in data:
        train_anno.append(each)
  print(len(train_anno))

  with open("data/coco/data/train_anno_LT.json", "w") as f:
    json.dump(train_anno, f)
  


82081 {'file_name': 'COCO_train2014_000000057870.jpg', 'labels': [12, 77, 51, 22, 27]}
1861
